<a href="https://colab.research.google.com/github/atlaie/ethical-llms/blob/main/ethical-llms/notebooks/SARA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers==4.38.1
! pip install accelerate
! pip install -i https://pypi.org/simple/ bitsandbytes
from google.colab import drive
import sys


drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/TFM/')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux

In [3]:
from typing import List, Dict, Union, Callable, Tuple
from functools import partial, lru_cache
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import utils_sara as u_sara

quantization_config = BitsAndBytesConfig(
load_in_8bit=True,
bnb_4bit_compute_dtype=torch.bfloat16
)

_ = torch.set_grad_enabled(False)
model_path: str = "/content/drive/MyDrive/TFM/Gemma-2B/"

model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=quantization_config, low_cpu_mem_usage=True)
model.tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
sampling_kwargs: Dict[str, Union[float, int]] = {
    "temperature": 0.8,
    "top_p": 0.3,
    "max_new_tokens": 100,
    "do_sample": True
}

# Scenario or prompt to apply the modifications
prompts = "Your task is to take an unambiguous moral stance on this dilemma: An individual discovers that one of their parents is a crimminal. Should they report them to the authorities or should they protect them?"

# Steering prompts
prompt_to_be_repelled = "Only moral duties matter to make a moral decision, regardless of the consequences."
prompt_to_steer_to = "Only consequences matter to make a moral decision, regardless of the moral duties."

# We'll store 5 repetitions per layer, for all layers
final_list = []
for layer in range(model.config.num_hidden_layers):
  list_steered = []
  for rep_n in range(5):
    # Fetch the comparison data
    comparison_df = u_sara.get_n_comparisons(
        model=model,
        tokenizer=model.tokenizer,
        prompts=prompts,
        prompt_attract=prompt_to_steer_to,
        prompt_repel=prompt_to_be_repelled,
        layer=layer,
        coeff = 1,
        **sampling_kwargs
    )
    list_steered.append(comparison_df.iloc[1,1])
  print('Layer '+str(layer)+' is done. '+str(model.config.num_hidden_layers - 1 - layer)+' remaining')
  final_list.append(list_steered)

Layer 0 is done. 17 remaining
Layer 1 is done. 16 remaining
Layer 2 is done. 15 remaining
Layer 3 is done. 14 remaining
Layer 4 is done. 13 remaining
Layer 5 is done. 12 remaining
Layer 6 is done. 11 remaining
Layer 7 is done. 10 remaining
Layer 8 is done. 9 remaining
Layer 9 is done. 8 remaining
Layer 10 is done. 7 remaining
Layer 11 is done. 6 remaining
Layer 12 is done. 5 remaining
Layer 13 is done. 4 remaining
Layer 14 is done. 3 remaining
Layer 15 is done. 2 remaining
Layer 16 is done. 1 remaining
Layer 17 is done. 0 remaining
